In [1]:
cd ../src

/storage2/home2/e1-313-15477/govind/s2slp/src


In [2]:
import data_preparer
import link_predictor

In [3]:
data_preparer.main()

100%|██████████| 10/10 [00:00<00:00, 15.63it/s]


In [4]:
link_predictor.main()

100%|██████████| 10/10 [00:02<00:00,  3.48it/s]

                  train            test
cn       0.5055+-0.0044   0.502+-0.0068
cn_      0.5057+-0.0044  0.5023+-0.0065
cn_mean  0.5056+-0.0044  0.5022+-0.0064
